In [24]:
import re
import pprint

In [25]:
with open('../boosted_bandit/bandit_results.db', 'r') as file:
    recs = file.readlines()

In [26]:
pos_recs = [rec for rec in recs if not rec.startswith('!')]
neg_recs = [rec[1:] for rec in recs if rec.startswith('!')]
all_recs = pos_recs + neg_recs

In [27]:
pattern = r'\d+\.?\d*'
items_and_probs = [tuple(re.findall(pattern, rec)) for rec in all_recs]

In [28]:
bev_items_probs = [rec for i, rec in enumerate(items_and_probs) if 'bev' in all_recs[i]]
food_items_probs = [rec for i, rec in enumerate(items_and_probs) if 'food' in all_recs[i]]

In [29]:
'''
bevs = {
1: [(item, prob), (item, prob), ...],
2: [],
...,
25: []
}
'''

'\nbevs = {\n1: [(item, prob), (item, prob), ...],\n2: [],\n...,\n25: []\n}\n'

In [30]:
user_bevs = {i:[] for i in range(1, 26)}
user_foods = {i:[] for i in range(1, 26)}

In [31]:
for user, bev, prob in bev_items_probs:
    user_bevs[int(user)].append((bev, float(prob)))
    
for user, food, prob in food_items_probs:
    user_foods[int(user)].append((food, float(prob)))

In [32]:
rec_user_bevs = {i:[] for i in range(1, 26)}
rec_user_foods = {i:[] for i in range(1, 26)}

In [33]:
for user, bevs in user_bevs.items():
    highest_prob = 0
    for bev, prob in bevs:
        if prob > highest_prob:
            highest_prob = prob
    highest_bevs = [bev for bev, prob in bevs if prob == highest_prob]
    rec_user_bevs[user] = highest_bevs

for user, foods in user_foods.items():
    highest_prob = 0
    for food, prob in foods:
        if prob > highest_prob:
            highest_prob = prob
    highest_foods = [food for food, prob in foods if prob == highest_prob]
    rec_user_foods[user] = highest_foods


In [35]:
rec_user_bevs

{1: ['22', '9'],
 2: ['21', '6', '20', '18', '10'],
 3: ['21', '1'],
 4: ['20', '21', '13', '7', '12', '10', '18', '4', '24'],
 5: ['2', '11'],
 6: ['5', '19', '25', '6', '24'],
 7: ['19', '11', '12', '18', '9', '13'],
 8: ['16', '24', '5', '19', '9'],
 9: ['3', '20', '25', '2'],
 10: ['14', '5', '3', '24', '8', '21', '4'],
 11: ['2', '21', '25', '19', '14', '15', '9'],
 12: ['23', '8', '5', '11', '4'],
 13: ['15', '19', '4'],
 14: ['1', '16', '6'],
 15: ['25', '15', '24', '5'],
 16: ['8', '11', '2', '3', '21', '25', '1'],
 17: ['13', '9', '2', '4'],
 18: ['18', '20', '24', '10', '1', '19'],
 19: ['21', '18', '16', '11', '12'],
 20: ['8', '20', '5', '14'],
 21: ['7', '24', '14', '5', '1', '19', '11'],
 22: ['9', '4', '21', '6', '15', '25', '18', '5', '20'],
 23: ['11', '19', '20'],
 24: ['8', '1', '9', '12', '6', '11'],
 25: ['5', '3', '4', '19', '25']}

# Reading User Data

In [15]:
import json
import random

In [23]:
for i in range(1, 26):
    user_info = open(f'../user_input_data/user_{i}.json', 'r')
    bevs = rec_user_bevs[i]
    foods = rec_user_bevs[i]
    
    rec = [{"day 1": [{"meal_name": "breakfast", "meal_time": "9:00", "Beverage": "", "Main Course": ""}, 
                      {"meal_name": "lunch", "meal_time": "13:00", "Beverage": "", "Main Course": "", "Side": ""}, 
                      {"meal_name": "dinner", "meal_time": "20:00", "Beverage": "6", "Main Course": "", "Dessert": "", "Side": ""}]}]
    for j, day in enumerate(rec, 1):
        day_rec = day[f'day {j}']
        for meal in day_rec:
            if "Beverage" in meal:
                meal["Beverage"] = random.choice(bevs)
            
            if "Main Course" in meal:
                meal["Main Course"] = random.choice(foods)
            
            if "Side" in meal:
                meal["Side"] = random.choice(foods)
            
            if "Dessert" in meal:
                meal["Dessert"] = random.choice(foods)
            
        
    rec = {"meal_plan":rec}
    with open(f'../recommendations/bandit_recs/recommendation_user_{i}.json', 'w') as file:
        json.dump(rec, file, indent=2)